### Step 8.  Summarization.  Pegasus.

### Import initial libraries and import the submissions CSV that includes sentiment scores

In [1]:
import pandas as pd
from datasets import Dataset
import re

Import submissions data and preview

In [3]:
tfcc_data = pd.read_csv('tfcc_submissions_top20_with_sentiment_including_comment_sentiment.csv')

tfcc_data.head()

,id,title,selftext,author,score,num_comments,created_date,selftext_length,topic,pos_sentiment,neg_sentiment,comments_pos_sentiment,comments_neg_sentiment
0,ydntm,"[I can't tell you] information line, this is P...",yay! 28th subscriber!\n\ni used to work for a...,PoglaTheGrate,29,4,2012-08-17 15:02:53,463,0,0.348745,0.651255,NaN,NaN
1,yicc4,Dishwasher blues,the stories i've heard amazes me but this one ...,[deleted],31,15,2012-08-20 05:05:34,431,11,0.000587,0.999413,0.011522,0.988478
2,yq9gg,Payroll company support?,we recently hired a payroll company at my comp...,IHopeYouStepOnALego,18,4,2012-08-24 01:28:44,205,2,0.002177,0.997823,NaN,NaN
3,zcvtt,Buildings can't be at fault,i work as a debt collector relating to car acc...,Xenomorphia,84,6,2012-09-04 21:45:59,275,5,0.001273,0.998727,NaN,NaN
4,zgoxq,Hurricanes,i was working at an online christian bookstore...,Ashleyrah,39,2,2012-09-06 19:11:23,154,2,0.001142,0.998858,NaN,NaN


Import comments data and preview

In [4]:
tfcc_comments = pd.read_csv('tfcc_comments_top20_with_sentiment.csv')

tfcc_comments

,parent_id,selftext,score,comments_pos_sentiment,comments_neg_sentiment
0,yicc4,I would like to find out who's idea it was to ...,6.0,0.001093,0.998907
1,yicc4,&gt;Verify bullshit\n\nWell actually would you...,1.0,0.021951,0.978049
2,10hg9t,Look for better paying work. Either you get a ...,1.0,0.517005,0.482995
3,12blfq,I work for customer retention for one of the b...,9.0,0.005717,0.994283
4,12blfq,I had an elderly man call up to cancel his wif...,6.0,0.000882,0.999118
...,...,...,...,...,...
9401,zzk31c,Your manager wasn't required to live monitor a...,13.0,0.000870,0.999130
9402,zzk31c,Depends on the centre and how well you're trea...,6.0,0.499343,0.500657
9403,zzk31c,I've actually done it a couple of times but it...,6.0,0.000823,0.999177
9404,zz8ztg,I've told the story on here before so I'll jus...,3.0,0.000564,0.999436


#### Import the comments CSV that includes sentiment scores

#### Select up to 125 submissions for each topic, using score as the ranking criteria

In [5]:
# sort the dataframe by topic and score in descending order
tfcc_data = tfcc_data.sort_values(by=["topic", "score"], ascending=[True, False])

# group the dataframe by topic and select the top 100 scores for each group
tfcc_data = tfcc_data.groupby("topic").head(125)

# reset the index of the dataframe
tfcc_data = tfcc_data.reset_index(drop=True)



In [6]:
tfcc_data

,id,title,selftext,author,score,num_comments,created_date,selftext_length,topic,pos_sentiment,neg_sentiment,comments_pos_sentiment,comments_neg_sentiment
0,9odgd4,"If you cuss before you tell me why, I'm hangin...","so. this just happened. i had a caller, we wil...",sleepernick,4183,167,2018-10-15 15:03:20,317,0,0.002438,0.997562,0.167575,0.832425
1,acncqg,I swear I’m not usually this dumb: when the ca...,my absolute favorite type of calls are when th...,QuoteTheKitty,2002,63,2019-01-04 22:13:45,173,0,0.360828,0.639172,0.239294,0.760706
2,js29up,The mute button is not the customers friend,at a previous call center i worked they wanted...,supersizedlady,1756,115,2020-11-11 05:30:44,240,0,0.003723,0.996277,0.305934,0.694066
3,e01rb8,One of my agents actually said what everyone t...,this happened a couple of weeks ago and is bot...,wirwarennamenlos,1553,121,2019-11-22 14:33:25,138,0,0.498418,0.501582,0.250270,0.749730
4,h0gvxc,Customer loves his analogy until I use it agai...,this call was from a while ago so i don't real...,BostonB96,1411,42,2020-06-10 18:28:42,330,0,0.184346,0.815654,0.001718,0.998282
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1959,4oloj6,Shooter threat,"wednesday night, someone called in and threate...",Believeinthis,18,20,2016-06-17 22:08:20,149,19,0.001418,0.998582,0.001219,0.998781
1960,2dbpxp,How come people assume call center employees a...,i've had a few infuriating calls. people just ...,lacquerqueen,15,21,2014-08-12 11:05:09,254,19,0.001012,0.998988,0.150063,0.849937
1961,4x4pjt,Probably getting a CA because I didn't apologi...,customer complained that i blew her off despi...,evosthunder,14,4,2016-08-10 22:06:34,104,19,0.013253,0.986747,NaN,NaN
1962,2ldeab,Don't Feed Stray Cats,"about two years ago, an old lady got upset and...",EveryoneHatesCJ,7,3,2014-11-05 15:11:42,125,19,0.005375,0.994625,NaN,NaN


In [7]:
filtered_df = tfcc_data.copy()

In [8]:
new_filtered_df = filtered_df.copy()

#### Initiate connection to Cohere

In [9]:
import cohere
import threading
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm
import time

co = cohere.Client("p4dcko27abVKk9AViHOCdvJM0KjyX3dmEfKA20Vy")

#### Function for retrieving summary from Cohere

In [18]:
def cohere_summary(text):
    attempts = 0
    while attempts < 3:
        try:
            response = co.summarize(
                model='summarize-xlarge',
                length='long',
                text=text
            )
            return response.summary
        except Exception as e:
            attempts += 1
            time.sleep(1)

    return None

def process_row(index, row, new_filtered_df):
    rtext = row["selftext"]
    summary = cohere_summary(rtext)
    
    if summary is not None:
        new_filtered_df.at[index, "summary"] = summary
    return summary is not None


#### Generate summaries and update dataframe

In [9]:
max_workers = 5

with ThreadPoolExecutor(max_workers=max_workers) as executor:
    tasks = []
    
    for index, row in filtered_df.iterrows():
        tasks.append(executor.submit(process_row, index, row, new_filtered_df))
    
    # Display progress using tqdm
    for task in tqdm(as_completed(tasks), total=len(tasks), desc="Processing rows"):
        pass


Processing rows: 100%|██████████| 1964/1964 [53:31<00:00,  1.64s/it]  


#### Display the first few rows and confirm summary has been added

In [10]:
new_filtered_df

,id,title,selftext,author,score,num_comments,created_date,selftext_length,topic,pos_sentiment,neg_sentiment,comments_pos_sentiment,comments_neg_sentiment,summary
0,9odgd4,"If you cuss before you tell me why, I'm hangin...","so. this just happened. i had a caller, we wil...",sleepernick,4183,167,2018-10-15 15:03:20,317,0,0.002438,0.997562,0.167575,0.832425,So I work in a call center. We have recorded l...
1,acncqg,I swear I’m not usually this dumb: when the ca...,my absolute favorite type of calls are when th...,QuoteTheKitty,2002,63,2019-01-04 22:13:45,173,0,0.360828,0.639172,0.239294,0.760706,My absolute favorite type of calls are when th...
2,js29up,The mute button is not the customers friend,at a previous call center i worked they wanted...,supersizedlady,1756,115,2020-11-11 05:30:44,240,0,0.003723,0.996277,0.305934,0.694066,I worked at a call center. We were not allowed...
3,e01rb8,One of my agents actually said what everyone t...,this happened a couple of weeks ago and is bot...,wirwarennamenlos,1553,121,2019-11-22 14:33:25,138,0,0.498418,0.501582,0.250270,0.749730,NaN
4,h0gvxc,Customer loves his analogy until I use it agai...,this call was from a while ago so i don't real...,BostonB96,1411,42,2020-06-10 18:28:42,330,0,0.184346,0.815654,0.001718,0.998282,This call was from a while ago so I don't real...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1959,4oloj6,Shooter threat,"wednesday night, someone called in and threate...",Believeinthis,18,20,2016-06-17 22:08:20,149,19,0.001418,0.998582,0.001219,0.998781,"Last week, someone called in a bomb threat to ..."
1960,2dbpxp,How come people assume call center employees a...,i've had a few infuriating calls. people just ...,lacquerqueen,15,21,2014-08-12 11:05:09,254,19,0.001012,0.998988,0.150063,0.849937,I work in a call center. I answer the phone. I...
1961,4x4pjt,Probably getting a CA because I didn't apologi...,customer complained that i blew her off despi...,evosthunder,14,4,2016-08-10 22:06:34,104,19,0.013253,0.986747,NaN,NaN,I work in a call center. Today I got chewed ou...
1962,2ldeab,Don't Feed Stray Cats,"about two years ago, an old lady got upset and...",EveryoneHatesCJ,7,3,2014-11-05 15:11:42,125,19,0.005375,0.994625,NaN,NaN,I work in a call center. It's a large company ...


#### Calculate the average length of the generated summaries

In [11]:
column = new_filtered_df['summary']

# compute the average number of words in the column values
avg_num_words = sum(len(str(value).split()) for value in column) / len(column)

In [12]:
avg_num_words

89.38085539714868

#### Export the dataframe with summaries to a CSV

In [13]:
new_filtered_df.to_csv('tfcc_submissions_top20_cohere_summaries.csv', index=False)

#### Add topic number to comments

In [10]:
# Extract a subset of tfcc_data with only id and topic columns and rename id column
tfcc_data_subset = tfcc_data[['id', 'topic']].rename(columns={'id': 'parent_id'})

# Merge tfcc_comments with tfcc_data_subset based on parent_id column
tfcc_comments_with_topics = pd.merge(tfcc_comments, tfcc_data_subset, on='parent_id', how='left')

# Keep the existing columns in tfcc_comments as well as the new topic column
columns_to_keep = ['parent_id', 'selftext', 'score', 'comments_pos_sentiment', 'comments_neg_sentiment', 'topic']
tfcc_comments_with_topics = tfcc_comments_with_topics[columns_to_keep]

In [11]:
tfcc_comments_with_topics

,parent_id,selftext,score,comments_pos_sentiment,comments_neg_sentiment,topic
0,yicc4,I would like to find out who's idea it was to ...,6.0,0.001093,0.998907,11.0
1,yicc4,&gt;Verify bullshit\n\nWell actually would you...,1.0,0.021951,0.978049,11.0
2,10hg9t,Look for better paying work. Either you get a ...,1.0,0.517005,0.482995,NaN
3,12blfq,I work for customer retention for one of the b...,9.0,0.005717,0.994283,10.0
4,12blfq,I had an elderly man call up to cancel his wif...,6.0,0.000882,0.999118,10.0
...,...,...,...,...,...,...
9401,zzk31c,Your manager wasn't required to live monitor a...,13.0,0.000870,0.999130,15.0
9402,zzk31c,Depends on the centre and how well you're trea...,6.0,0.499343,0.500657,15.0
9403,zzk31c,I've actually done it a couple of times but it...,6.0,0.000823,0.999177,15.0
9404,zz8ztg,I've told the story on here before so I'll jus...,3.0,0.000564,0.999436,NaN


#### Reduce size of dataset to summarize by limiting up to 100 comments per topic based on score

In [12]:

# Group the dataframe by topic and apply a function to keep only the top 100 rows based on score
import pandas as pd

# sort the dataframe by topic and score in descending order
tfcc_comments_with_topics = tfcc_comments_with_topics.sort_values(['topic', 'score'], ascending=[True, False])

# group the dataframe by topic and keep the top 100 rows for each group
tfcc_comments_with_topics = tfcc_comments_with_topics.groupby('topic').head(100)

# print the updated dataframe
print(tfcc_comments_with_topics)

     parent_id                                           selftext  score  \
3432    9odgd4  Where I work, we're supposed to somewhat lay d...  618.0   
6564    js29up  I heard a customer saying that his girl needs ...  518.0   
3745    adu7ng  Jay-Z wanted a free iPhone 5 when it came out....  442.0   
7158    loeg8v  We've all had this guy. In this this guy's cas...  402.0   
6428    j65aiy  As a female who deals with clients 95% over em...  399.0   
...        ...                                                ...    ...   
7084    ld7gls  When I worked in collections, I heard that a l...    5.0   
7335    mh74wj  This reminds me of ome of my jobs. My boss and...    5.0   
7699    ob5rqm  You are not alone. \n\nThis seems very typical...    5.0   
8162    qma91m  Just curiouswhen the get to cancellations does...    5.0   
8286    rhp1cg  You fuckin' moron! Dont you know how much I fu...    5.0   

      comments_pos_sentiment  comments_neg_sentiment  topic  
3432                0.000

#### Display the number of comments by topic

In [13]:
# Group the DataFrame by the topic column and count the number of entries in each group
num_entries_by_topic = tfcc_comments_with_topics.groupby('topic').size().reset_index(name='num_entries')

print(num_entries_by_topic)

    topic  num_entries
0     0.0          100
1     1.0          100
2     2.0          100
3     3.0          100
4     4.0          100
5     5.0          100
6     6.0          100
7     7.0          100
8     8.0          100
9     9.0          100
10   10.0          100
11   11.0          100
12   12.0          100
13   13.0          100
14   14.0          100
15   15.0          100
16   16.0          100
17   17.0           92
18   18.0          100
19   19.0          100


#### Combine the combines into large chunks for summarization

In [14]:
# Group the tfcc_comments_with_topics DataFrame by the 'topic' column
grouped = tfcc_comments_with_topics.groupby('topic')

# Initialize the new DataFrame with empty lists
new_df = pd.DataFrame({'topic': [], 'selftext': []})

# Iterate through each group in the grouped DataFrame
for group_name, group_df in grouped:
    # Concatenate the 'selftext' values in the group with '~~' separator
    selftext_list = group_df['selftext'].tolist()
    selftext_concatenated = ''
    for selftext in selftext_list:
        # Check if the concatenated selftext would exceed 2000 words
        words = selftext_concatenated.split()
        if len(words) + len(selftext.split()) > 2000:
            # Add the current concatenated selftext to the new DataFrame
            new_df = new_df.append({'topic': group_name, 'selftext': selftext_concatenated.strip()}, ignore_index=True)
            selftext_concatenated = ''
        # Concatenate the current selftext to the concatenated selftext with '~~' separator
        selftext_concatenated += selftext + '~~'
    # Add any remaining concatenated selftext to the new DataFrame
    new_df = new_df.append({'topic': group_name, 'selftext': selftext_concatenated.strip()}, ignore_index=True)

/tmp/ipykernel_23998/2844121060.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append({'topic': group_name, 'selftext': selftext_concatenated.strip()}, ignore_index=True)
/tmp/ipykernel_23998/2844121060.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append({'topic': group_name, 'selftext': selftext_concatenated.strip()}, ignore_index=True)
/tmp/ipykernel_23998/2844121060.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append({'topic': group_name, 'selftext': selftext_concatenated.strip()}, ignore_index=True)
/tmp/ipykernel_23998/2844121060.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use panda

#### Save progress to CSV

In [15]:
new_df.to_csv('tfcc_top_comments_grouped_by_topic_combined_cohere.csv', index=False)

In [16]:
comments_df = pd.read_csv('tfcc_top_comments_grouped_by_topic_combined_cohere.csv')


#### Summarize the comments chunks

In [19]:
max_workers = 5

with ThreadPoolExecutor(max_workers=max_workers) as executor:
    tasks = []
    
    for index, row in comments_df.iterrows():
        tasks.append(executor.submit(process_row, index, row, comments_df))
    
    # Display progress using tqdm
    for task in tqdm(as_completed(tasks), total=len(tasks), desc="Processing rows"):
        pass



Processing rows: 100%|██████████| 99/99 [05:49<00:00,  3.53s/it]


In [20]:
comments_df

,topic,selftext,summary
0,0.0,"Where I work, we're supposed to somewhat lay d...",When a customer called a company asking for a ...
1,0.0,"Not sure why I was reminded of this, but my gr...",I'm not sure if this is a thing in other count...
2,0.0,"""Oh, mensa. Isn't that the private organizatio...",A customer called a company to complain about ...
3,0.0,Oh thats awful. Im assuming there were rules a...,I'm not sure how to start this. I work in a ca...
4,0.0,My older brother had this moment when we were ...,My older brother had this moment when we were ...
...,...,...,...
94,19.0,Buck up and tell them to get their shit togeth...,I work in a call center. People are always thr...
95,19.0,"I don't remember the context, but somebody was...",I've worked in a call center for over 10 years...
96,19.0,"So true, I had to call a customer back yesterd...",I'm sure we all have our own stories to tell a...
97,19.0,"From the other side of the fence, I just had a...",I'm sure there are many people who have had to...


In [21]:
# Group the DataFrame by the topic column and count the number of entries in each group
num_entries_by_topic = comments_df.groupby('topic').size().reset_index(name='num_entries')

print(num_entries_by_topic)

    topic  num_entries
0     0.0            5
1     1.0            5
2     2.0            5
3     3.0            5
4     4.0            5
5     5.0            5
6     6.0            5
7     7.0            5
8     8.0            5
9     9.0            5
10   10.0            5
11   11.0            5
12   12.0            5
13   13.0            5
14   14.0            5
15   15.0            5
16   16.0            5
17   17.0            4
18   18.0            5
19   19.0            5


#### Export the comments summaries to a csv

In [22]:
comments_df.to_csv('tfcc_top_comments_cohere_summarized.csv', index=False)